<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/%EC%BD%9C%EA%B8%88%EB%A6%AC_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import requests

no_page = 693
dates =[]
call_rates = []
index = ['date', 'callrate']

for page in range(no_page):

  url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn'
  query = {
      'marketindexCd' : 'IRR_CALL',
      'page' : page+1
  }
  resp = requests.get(url, query)
  soup = BeautifulSoup(resp.content, 'lxml')

  date = soup.select('td.date')
  call_rate = soup.select('td.num')
  for i in range(len(date)):
    dates.append(date[i].text.strip())
    call_rates.append(call_rate[i*3-3].text.strip())
Call_rates = pd.DataFrame([dates, call_rates], index=index)
Call_rates = Call_rates.T
Call_rates.to_csv('Call_rates.csv')
#naver callrate crawl, transpose, save

In [ ]:
Call_rates

,date,callrate
0,2020.07.30,0.45
1,2020.07.29,0.49
2,2020.07.28,0.48
3,2020.07.27,0.47
4,2020.07.24,0.49
...,...,...
4846,2001.01.12,5.30
4847,2001.01.11,5.33
4848,2001.01.10,5.32
4849,2001.01.09,5.32


In [ ]:
import pandas
import numpy as np

dt_index = pandas.date_range(start='20041201', end='20171231')

dt_list = dt_index.strftime("%Y.%m.%d").tolist()
datels = []
for i in dt_list:
    datels.append(i)
datels
#to make 'None' cells, use pandas

['2004.12.01',
 '2004.12.02',
 '2004.12.03',
 '2004.12.04',
 '2004.12.05',
 '2004.12.06',
 '2004.12.07',
 '2004.12.08',
 '2004.12.09',
 '2004.12.10',
 '2004.12.11',
 '2004.12.12',
 '2004.12.13',
 '2004.12.14',
 '2004.12.15',
 '2004.12.16',
 '2004.12.17',
 '2004.12.18',
 '2004.12.19',
 '2004.12.20',
 '2004.12.21',
 '2004.12.22',
 '2004.12.23',
 '2004.12.24',
 '2004.12.25',
 '2004.12.26',
 '2004.12.27',
 '2004.12.28',
 '2004.12.29',
 '2004.12.30',
 '2004.12.31',
 '2005.01.01',
 '2005.01.02',
 '2005.01.03',
 '2005.01.04',
 '2005.01.05',
 '2005.01.06',
 '2005.01.07',
 '2005.01.08',
 '2005.01.09',
 '2005.01.10',
 '2005.01.11',
 '2005.01.12',
 '2005.01.13',
 '2005.01.14',
 '2005.01.15',
 '2005.01.16',
 '2005.01.17',
 '2005.01.18',
 '2005.01.19',
 '2005.01.20',
 '2005.01.21',
 '2005.01.22',
 '2005.01.23',
 '2005.01.24',
 '2005.01.25',
 '2005.01.26',
 '2005.01.27',
 '2005.01.28',
 '2005.01.29',
 '2005.01.30',
 '2005.01.31',
 '2005.02.01',
 '2005.02.02',
 '2005.02.03',
 '2005.02.04',
 '2005.02.

In [ ]:
datels = datels[::-1]
dfno = pd.DataFrame([datels], index=['date'])
dfno = dfno.T
dfno

,date
0,2017.12.31
1,2017.12.30
2,2017.12.29
3,2017.12.28
4,2017.12.27
...,...
4774,2004.12.05
4775,2004.12.04
4776,2004.12.03
4777,2004.12.02


In [ ]:
k = pd.merge(dfno, Call_rates, how='left')
k

,date,callrate
0,2017.12.31,NaN
1,2017.12.30,NaN
2,2017.12.29,1.52
3,2017.12.28,1.50
4,2017.12.27,1.55
...,...,...
4774,2004.12.05,NaN
4775,2004.12.04,NaN
4776,2004.12.03,3.26
4777,2004.12.02,3.26


In [ ]:
Call = k.fillna(method='bfill')         # filling None cells with value of cell before them.. if the last cell's value was 'None', it wouecome moreld have b complicated job.

In [ ]:
Call

,date,callrate
0,2017.12.31,1.52
1,2017.12.30,1.52
2,2017.12.29,1.52
3,2017.12.28,1.50
4,2017.12.27,1.55
...,...,...
4774,2004.12.05,3.26
4775,2004.12.04,3.26
4776,2004.12.03,3.26
4777,2004.12.02,3.26


In [ ]:
Call_A = Call.loc[:][:][:-31]
Call_B = Call.loc[:][:][31:].rename({'date':'30일전 date', 'callrate':'30일전 callrate'}, axis='columns').reset_index(drop=True)

#30 days = one month

In [ ]:
len(Call_A), len(Call_B)

(4748, 4748)

In [ ]:
CALLRATE = pd.merge(Call_A, Call_B, left_index=True, right_index=True)
# CALLRATE['한달전 date'][0], CALLRATE['한달전 callrate'][0] = CALLRATE['한달전 date'][1], CALLRATE['한달전 callrate'][1]# 12/31일이 그냥 슬라이싱 한거로 가져오면 12/01을 가져오게 되기 때문에 마지막 날만 맞춰줌... 근데 숫자만 31일 전으로 돌리는것 같아서 뭔가 잘못된것같네...다시해야겠다

CALLRATE#한달전을 30일전으로 따지기로 했음. 이유- 이미 많은 주말들을 fillna로 채워줬다. 

,date,callrate,30일전 date,30일전 callrate
0,2017.12.31,1.52,2017.11.30,1.48
1,2017.12.30,1.52,2017.11.29,1.53
2,2017.12.29,1.52,2017.11.28,1.25
3,2017.12.28,1.50,2017.11.27,1.26
4,2017.12.27,1.55,2017.11.26,1.26
...,...,...,...,...
4743,2005.01.05,3.21,2004.12.05,3.26
4744,2005.01.04,3.28,2004.12.04,3.26
4745,2005.01.03,3.29,2004.12.03,3.26
4746,2005.01.02,3.27,2004.12.02,3.26


In [ ]:
# from datetime import date

# day = date(2015, 3, 28)
# time = day - pd.DateOffset(months=1)
# print(time.date())
# 한달전 구하는 방법, 적용이 힘들어서 안쓰기로 함

In [ ]:
CALLRATE['callrate']=CALLRATE['callrate'].astype(float)
CALLRATE['30일전 callrate']=CALLRATE['30일전 callrate'].astype(float)

In [ ]:
CALLRATE['Polarity']=np.where(CALLRATE['callrate']>CALLRATE['30일전 callrate'],'H',np.where(CALLRATE['callrate']==CALLRATE['30일전 callrate'],'N','D'))

In [ ]:
CALLRATE

,date,callrate,30일전 date,30일전 callrate,Polarity
0,2017.12.31,1.52,2017.11.30,1.48,H
1,2017.12.30,1.52,2017.11.29,1.53,D
2,2017.12.29,1.52,2017.11.28,1.25,H
3,2017.12.28,1.50,2017.11.27,1.26,H
4,2017.12.27,1.55,2017.11.26,1.26,H
...,...,...,...,...,...
4743,2005.01.05,3.21,2004.12.05,3.26,D
4744,2005.01.04,3.28,2004.12.04,3.26,H
4745,2005.01.03,3.29,2004.12.03,3.26,H
4746,2005.01.02,3.27,2004.12.02,3.26,H


In [ ]:
datelabel=['date', 'Polarity']
Callrate_labeling = CALLRATE[datelabel]
Callrate_labeling

,date,Polarity
0,2017.12.31,H
1,2017.12.30,D
2,2017.12.29,H
3,2017.12.28,H
4,2017.12.27,H
...,...,...
4743,2005.01.05,D
4744,2005.01.04,H
4745,2005.01.03,H
4746,2005.01.02,H


In [ ]:
Callrate_labeling = Callrate_labeling.set_index('date')

In [ ]:
Callrate_labeling

,Polarity
date,
2017.12.31,H
2017.12.30,D
2017.12.29,H
2017.12.28,H
2017.12.27,H
...,...
2005.01.05,D
2005.01.04,H
2005.01.03,H


In [ ]:
Callrate_labeling.to_csv('Callrate_labeling.csv')

In [ ]:
Callrate_labeling.loc['2010.01.01']

Polarity    D
Name: 2010.01.01, dtype: object

In [ ]:
Cal = CALLRATE[:10]
Cal

,date,callrate,30일전 date,30일전 callrate,Polarity
0,2017.12.31,1.52,2017.11.30,1.48,H
1,2017.12.30,1.52,2017.11.29,1.53,D
2,2017.12.29,1.52,2017.11.28,1.25,H
3,2017.12.28,1.50,2017.11.27,1.26,H
4,2017.12.27,1.55,2017.11.26,1.26,H
5,2017.12.26,1.51,2017.11.25,1.26,H
6,2017.12.25,1.52,2017.11.24,1.26,H
7,2017.12.24,1.52,2017.11.23,1.25,H
8,2017.12.23,1.52,2017.11.22,1.25,H
9,2017.12.22,1.52,2017.11.21,1.24,H


In [ ]:
Cal[['date', 'callrate', 'Polarity']]

,date,callrate,Polarity
0,2017.12.31,1.52,H
1,2017.12.30,1.52,D
2,2017.12.29,1.52,H
3,2017.12.28,1.50,H
4,2017.12.27,1.55,H
5,2017.12.26,1.51,H
6,2017.12.25,1.52,H
7,2017.12.24,1.52,H
8,2017.12.23,1.52,H
9,2017.12.22,1.52,H


#콜금리 데이터프레임과 비교하여 들어온 df의 날짜를 보고 라벨링 하는 함수- Abandonded


In [ ]:
# caca=Cal[['date','callrate']].copy()

In [ ]:
Callrate_labeling = pd.read_csv('Callrate_labeling.csv')
본인파일.mergy(Callrate_labeling, on='date')
# Callrate_labeling = Callrate_labeling.set_index('date')

NameError: ignored

In [ ]:
def compare_append(txt):
  
  txt['Polarity'] = ''
  Hawkish = []
  Dovish = []
  Grey = []
  for i in range(len(txt)):
    temp_date = txt['date'][i]
    if Callrate_labeling.loc[str(temp_date)][0] == 'up':
      Hawkish.append(txt['contents'][i])
      txt['Polarity'][i] = 'H'
    elif Callrate_labeling.loc[str(temp_date)][0] == 'down':
      Dovish.append(txt['contents'][i])
      txt['Polarity'][i]='D'
    else:
      Grey.append(txt['contents'][i])
      txt['Polarity'][i]='N'
  Labeled = txt
  return Labeled, 'Hakish:', Hawkish, 'Dovish: ', Dovish, 'Grey: ',Grey#Labled를 따로 받을 수 있게 해주자

In [ ]:
# Callrate_labeling[:10]

In [ ]:
compare_append(Cal)

# 최종: 그냥 merge해주기로 했음

In [ ]:
Callrate_labeling = pd.read_csv('Callrate_labeling.csv')
본인파일.merge(Callrate_labeling, on='date')